```--- Day 15: Beacon Exclusion Zone ---
You feel the ground rumble again as the distress signal leads you to a large network of subterranean tunnels. You don't have time to search them all, but you don't need to: your pack contains a set of deployable sensors that you imagine were originally built to locate lost Elves.

The sensors aren't very powerful, but that's okay; your handheld device indicates that you're close enough to the source of the distress signal to use them. You pull the emergency sensor system out of your pack, hit the big button on top, and the sensors zoom off down the tunnels.

Once a sensor finds a spot it thinks will give it a good reading, it attaches itself to a hard surface and begins monitoring for the nearest signal source beacon. Sensors and beacons always exist at integer coordinates. Each sensor knows its own position and can determine the position of a beacon precisely; however, sensors can only lock on to the one beacon closest to the sensor as measured by the Manhattan distance. (There is never a tie where two beacons are the same distance to a sensor.)

It doesn't take long for the sensors to report back their positions and closest beacons (your puzzle input). For example:

Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3
So, consider the sensor at 2,18; the closest beacon to it is at -2,15. For the sensor at 9,16, the closest beacon to it is at 10,16.

Drawing sensors as S and beacons as B, the above arrangement of sensors and beacons looks like this:

               1    1    2    2
     0    5    0    5    0    5
 0 ....S.......................
 1 ......................S.....
 2 ...............S............
 3 ................SB..........
 4 ............................
 5 ............................
 6 ............................
 7 ..........S.......S.........
 8 ............................
 9 ............................
10 ....B.......................
11 ..S.........................
12 ............................
13 ............................
14 ..............S.......S.....
15 B...........................
16 ...........SB...............
17 ................S..........B
18 ....S.......................
19 ............................
20 ............S......S........
21 ............................
22 .......................B....
This isn't necessarily a comprehensive map of all beacons in the area, though. Because each sensor only identifies its closest beacon, if a sensor detects a beacon, you know there are no other beacons that close or closer to that sensor. There could still be beacons that just happen to not be the closest beacon to any sensor. Consider the sensor at 8,7:

               1    1    2    2
     0    5    0    5    0    5
-2 ..........#.................
-1 .........###................
 0 ....S...#####...............
 1 .......#######........S.....
 2 ......#########S............
 3 .....###########SB..........
 4 ....#############...........
 5 ...###############..........
 6 ..#################.........
 7 .#########S#######S#........
 8 ..#################.........
 9 ...###############..........
10 ....B############...........
11 ..S..###########............
12 ......#########.............
13 .......#######..............
14 ........#####.S.......S.....
15 B........###................
16 ..........#SB...............
17 ................S..........B
18 ....S.......................
19 ............................
20 ............S......S........
21 ............................
22 .......................B....
This sensor's closest beacon is at 2,10, and so you know there are no beacons that close or closer (in any positions marked #).

None of the detected beacons seem to be producing the distress signal, so you'll need to work out where the distress beacon is by working out where it isn't. For now, keep things simple by counting the positions where a beacon cannot possibly be along just a single row.

So, suppose you have an arrangement of beacons and sensors like in the example above and, just in the row where y=10, you'd like to count the number of positions a beacon cannot possibly exist. The coverage from all sensors near that row looks like this:

                 1    1    2    2
       0    5    0    5    0    5
 9 ...#########################...
10 ..####B######################..
11 .###S#############.###########.
In this example, in the row where y=10, there are 26 positions where a beacon cannot be present.

Consult the report from the sensors you just deployed. In the row where y=2000000, how many positions cannot contain a beacon?

```

In [1]:
import re
import numpy as np
from tqdm.auto import tqdm

data_raw = open('inputs/15.input').read()
data = np.array(list(map(int,re.findall(r'[yx]=(-?\d+)', data_raw)))).reshape(-1,2,2)

xmin,xmax,ymin,ymax = np.quantile(data,(0,1),axis=(0,1)).T.flatten()
sensors_radius = np.abs(data[:,0]-data[:,1]).sum(1)
max_radius = sensors_radius.max()
beacons_masks = np.isin(np.arange(data.shape[0]), np.unique(data[:,1],axis=0, return_index=True)[1])
beacons = data[beacons_masks,1]

In [2]:
def negative_in_row(row):
    segments = []
    for (x,y),r in zip(data[:,0],sensors_radius):
        if (dist:=abs(y-row)) > r: continue
        w2 = abs(r-dist)
        segments.append((x-w2,x+w2))
    return sorted(segments)

def count_segments(row,low_limit=-1e9,high_limit=1e9):
    total = 0 
    last_to = low_limit
    limits_list = [low_limit,low_limit]
    segments = negative_in_row(row)
    for s_from,s_to in segments:
        s_to = min(s_to,high_limit)
        
        if s_from > limits_list[-1]: limits_list += [1e10,-1e10]
        limits_list[-2] = min(limits_list[-2],max(s_from,low_limit))
        limits_list[-1] = max(limits_list[-1],s_to)
        
        w = s_to - s_from + 1
        i = min(max(0, last_to - s_from + 1),w)
        total += w - i
        last_to = max(s_to, last_to)
    return total, limits_list

def count_bs(row):
    bs_count = np.logical_or(data[:,0,1]==row,np.logical_and(data[:,1,1]==row,   beacons_masks)).sum()
    return bs_count

def count_row(row,*args,**kwargs):
    bs_count = count_bs(row)
    c,l = count_segments(row,*args,**kwargs)
    return c-bs_count, l

row = 2000000
count_row(row)[0]

4665948

```--- Part Two ---
Your handheld device indicates that the distress signal is coming from a beacon nearby. The distress beacon is not detected by any sensor, but the distress beacon must have x and y coordinates each no lower than 0 and no larger than 4000000.

To isolate the distress beacon's signal, you need to determine its tuning frequency, which can be found by multiplying its x coordinate by 4000000 and then adding its y coordinate.

In the example above, the search space is smaller: instead, the x and y coordinates can each be at most 20. With this reduced search area, there is only a single position that could have a beacon: x=14, y=11. The tuning frequency for this distress beacon is 56000011.

Find the only possible position for the distress beacon. What is its tuning frequency?
```

In [3]:
low_limit = 0
high_limit = 4000000
span = high_limit-low_limit+1

mult=4000000

for i in tqdm(range(span)):
    c,l = count_segments(i,low_limit=low_limit-1,high_limit=high_limit)
    
    if c != span:
        y = i
        x = int(sum(l[1:3])/2)
        print(x*mult+y)
        break

  0%|          | 0/4000001 [00:00<?, ?it/s]

13543690671045
